# Similarity Evaluation Analysis (SEA) Dataset E

In [1]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
print('Libraries imported!!')

Libraries imported!!


In [2]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = 'EVALUATION FUNCTIONS/PRIVACY'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)

#import functions for univariate resemblance analisys
from similarity_evaluation import scale_data
from similarity_evaluation import pairwise_euclidean_distance
from similarity_evaluation import hausdorff_distance
from similarity_evaluation import rts_similarity

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read real and synthetic datasets
In this part real and synthetic datasets are read.

In [3]:
#Define global variables
DATA_TYPES = ['Real','GM','SDV','CTGAN','WGANGP']
SYNTHESIZERS = ['GM','SDV','CTGAN','WGANGP']
FILEPATHS = {'Real' : HOME_PATH + 'REAL DATASETS/TRAIN DATASETS/E_PimaIndiansDiabetes_Real_Train.csv',
            'GM' : HOME_PATH + 'SYNTHETIC DATASETS/GM/E_PimaIndiansDiabetes_Synthetic_GM.csv',
            'SDV' : HOME_PATH + 'SYNTHETIC DATASETS/SDV/E_PimaIndiansDiabetes_Synthetic_SDV.csv',
            'CTGAN' : HOME_PATH + 'SYNTHETIC DATASETS/CTGAN/E_PimaIndiansDiabetes_Synthetic_CTGAN.csv',
            'WGANGP' : HOME_PATH + 'SYNTHETIC DATASETS/WGANGP/E_PimaIndiansDiabetes_Synthetic_WGANGP.csv'}
categorical_columns = ['Outcome']
data = dict()

In [5]:
#iterate over all datasets filepaths and read each dataset
for name, path in FILEPATHS.items() :
    data[name] = pd.read_csv(path)
    for col in categorical_columns :
        data[name][col] = data[name][col].astype('category').cat.codes
data

{'Real':      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
 0              2       84              0              0        0   0.0   
 1              9      112             82             24        0  28.2   
 2              1      139             46             19       83  28.7   
 3              0      161             50              0        0  21.9   
 4              6      134             80             37      370  46.2   
 ..           ...      ...            ...            ...      ...   ...   
 609            5      139             64             35      140  28.6   
 610            1       96            122              0        0  22.4   
 611           10      101             86             37        0  45.6   
 612            0      141              0              0        0  42.4   
 613            0      125             96              0        0  22.5   
 
      DiabetesPedigreeFunction  Age  Outcome  
 0                       0.304   21        

## 2. Normalize data

In [6]:
#Scale the data
num_cols = (data['Real'].select_dtypes(include=['int64','float64'])).columns
scaled_data = dict()
for name in DATA_TYPES :
    scaled_data[name] = scale_data(data[name])
scaled_data

{'Real':      Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
 0       0.117647  0.422111       0.000000       0.000000  0.000000  0.000000   
 1       0.529412  0.562814       0.672131       0.380952  0.000000  0.420268   
 2       0.058824  0.698492       0.377049       0.301587  0.098109  0.427720   
 3       0.000000  0.809045       0.409836       0.000000  0.000000  0.326379   
 4       0.352941  0.673367       0.655738       0.587302  0.437352  0.688525   
 ..           ...       ...            ...            ...       ...       ...   
 609     0.294118  0.698492       0.524590       0.555556  0.165485  0.426230   
 610     0.058824  0.482412       1.000000       0.000000  0.000000  0.333830   
 611     0.588235  0.507538       0.704918       0.587302  0.000000  0.679583   
 612     0.000000  0.708543       0.000000       0.000000  0.000000  0.631893   
 613     0.000000  0.628141       0.786885       0.000000  0.000000  0.335320   
 
      DiabetesPedi

## 3. Calculate the Euclidean distances between each pair of values

In [7]:
distances_values = dict()

for name in SYNTHESIZERS :
    #distances = distance.cdist(scaled_data[name].values, real_data_scaled, 'euclidean')
    distances_values[name] = pairwise_euclidean_distance(scaled_data[name].values, scaled_data['Real'].values)
    
distances_values

{'GM': '1.0291 ± 0.3182',
 'SDV': '1.0872 ± 0.2921',
 'CTGAN': '0.993 ± 0.3178',
 'WGANGP': '1.0973 ± 0.2658'}

## 4. Calculate the Hausdorff distance between synthetic data and real data

In [8]:
hausdorff_values = dict()

for name in SYNTHESIZERS :
    hausdorff_values[name] = hausdorff_distance(scaled_data[name].values, scaled_data['Real'].values)
    
hausdorff_values

{'GM': 0.723, 'SDV': 0.8446, 'CTGAN': 0.7578, 'WGANGP': 0.9053}

## 5. Calculate maximum RTS similarity

In [9]:
str_values = dict()

for name in SYNTHESIZERS :
    str_values[name] = rts_similarity(scaled_data[name].values, scaled_data['Real'].values)
    
str_values

{'GM': {'min': 0.0204, 'mean': 0.7528, 'max': 0.9969},
 'SDV': {'min': 0.0766, 'mean': 0.731, 'max': 0.9944},
 'CTGAN': {'min': 0.052, 'mean': 0.7492, 'max': 0.9976},
 'WGANGP': {'min': 0.0213, 'mean': 0.7426, 'max': 0.9895}}